In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



#postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
#postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
#avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
#avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')



In [ ]:
avisosDetalleZIP = pd.read_csv('resources/zip/fiuba_6_avisos_detalle.csv')
avisosDetalleMissing = pd.read_csv('resources/zip/fiuba_6_avisos_missing.csv')

avisosDetallesCompletos = pd.concat([avisosDetalleZIP,avisosDetalleMissing])

### Vistas 

In [ ]:
vistas['timestamp'] = pd.to_datetime(vistas['timestamp'])
vistas['dateVista'] = pd.DatetimeIndex(vistas['timestamp']).date

vistasGrouped = vistas.groupby(['dateVista','idpostulante','idAviso']).count();
vistasGrouped = vistasGrouped.reset_index();

vistasGrouped['cantidadVistas'] = vistasGrouped['timestamp'];
vistasGrouped.drop('timestamp',axis=1,inplace=True)
vistasForMerge = vistasGrouped[['idpostulante','idAviso']]


### Postulaciones 

In [ ]:
avisosPostulados['cantPostulaciones'] = 1;
avisosPostuladosGP = avisosPostulados.groupby(['idaviso','idpostulante']).sum()
avisosPostuladosGP = avisosPostuladosGP.reset_index()
avisosPostuladosGP.info

In [ ]:
import datetime as dt

def generoNOAvisos(cantParaArmar):
    sumaNoPostulados = pd.DataFrame();
    for i in range(int(cantParaArmar)):
        print("--------------------- Iteracion de sample " + str(i) +" de "+ str(cantParaArmar) +"---------");
        vistasSample = vistasForMerge.sample(frac=0.1);
        postulacionesSample = avisosPostuladosGP.sample(frac=0.1);

        avisosNoVistos = pd.merge(postulacionesSample,vistasSample,left_on='idaviso',right_on='idAviso',how='right', indicator=True)
        avisosNoVistosPosta = avisosNoVistos[avisosNoVistos['_merge'] == 'right_only']
#avisosNoVistosFinal = avisosNoVistosPosta.drop(['idpostulante_x','datePostulacion','cantPostulaciones','_merge'],axis=1)
        avisosNoVistosFinal = avisosNoVistosPosta.drop(['idpostulante_x','cantPostulaciones','_merge'],axis=1)
        avisosNoVistosFinal['idaviso'] = avisosNoVistosFinal['idAviso'];
        avisosNoVistosFinal['idpostulante'] = avisosNoVistosFinal['idpostulante_y'];
        avisosNoVistosFinal.drop(['idpostulante_y','idAviso'],axis=1,inplace=True);
        sumaNoPostulados = pd.concat([sumaNoPostulados,avisosNoVistosFinal])
    return sumaNoPostulados;


def generoArchivosNOPostulado(cantArchivos,cantInicial,cantIteracionesPorArhivo):
    print("--- HORA DE INICIO PROCESO TOTAL "+ str(dt.datetime.now()) + "-----");
    for i in range(int(cantInicial),int(cantArchivos)):
        print("---------- Generando archivos " + str(i) +" de "+ str(cantArchivos) +"-------");
        print("---------- HORA DE INICIO "+ str(dt.datetime.now()) + "-------");
        sumaTotalNoAvisos = generoNOAvisos(int(cantIteracionesPorArhivo));
        sumaTotalNoAvisosSinDuplicados = sumaTotalNoAvisos.drop_duplicates().reset_index()
        sumaTotalNoAvisosSinDuplicados['sepostulo'] = 0
        sumaTotalNoAvisosSinDuplicados = sumaTotalNoAvisosSinDuplicados[['idpostulante','idaviso','sepostulo']]
        sumaTotalNoAvisosSinDuplicados.to_csv('nopostulados/nopostulados-'+str(i)+'.csv');
        print("---------- HORA DE FIN "+ str(dt.datetime.now()) + "--------------------------");
    print("--- HORA DE FIN PROCESO TOTAL "+ str(dt.datetime.now()) + "-----");
        

In [ ]:
##Llamamos proceso

generoArchivosNOPostulado(15,10,20);

# Proceso II

In [3]:
vistasChiquito = vistas[['idpostulante','idAviso']]
vistasChiquito['idaviso'] = vistas['idAviso']
vistasChiquito.drop('idAviso',axis=1,inplace=True)

pf = postulaciones[['idpostulante']].drop_duplicates()

In [ ]:
noPostuladosTotal = pd.DataFrame();
for i in range(0,238482,5000):
    print("----" + str(i) + "-----")
    pfRows = pf.iloc[i:i+5000]
    avisosConPostulacion = pd.merge(postulaciones,pfRows,on='idpostulante',how='inner')
    todos = pd.merge(vistasChiquito,avisosConPostulacion,on='idaviso',how='left')       
    noPostuTodos = todos[todos['idpostulante_y'].isnull()];
    noPostuFiltrados = noPostuTodos.sample(n=10);
    noPostuFiltrados = noPostuTodos;
    noPostuladosTotal = pd.concat([noPostuFiltrados,noPostuladosTotal])

   
noPostuladosTotal.to_csv('nopostulados/noPostuladosNG.csv')